<a href="https://colab.research.google.com/github/Rosie-Brigham/colab-gists/blob/main/Model_3c_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '/content/drive/MyDrive/all_submissions_01_02_21'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
model = torchvision.models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
# Here the size of each output sample is set to length of the class name list.
model.fc = nn.Linear(num_ftrs, len(class_names))

In [ ]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.cuda()
            y = y.cuda()
            model.cuda()
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
        
        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}') 
    
    model.train()



In [ ]:
def print_accuracy(dataloaders, model, name):
  print(f"Checking accuracy on Training Set - {name}")
  check_accuracy(dataloaders['train'], model)

  print(f"Checking accuracy on Test Set {name}")
  check_accuracy(dataloaders['val'], model)



Start model analysis

In [ ]:
path = "/content/drive/MyDrive/models/model_6_25e_conv.pt"
model.load_state_dict(torch.load(path))
model.eval()

print_accuracy(dataloaders, model, "Conv 25e")

Checking accuracy on Training Set - Conv 25e
Got 3076 / 3463 with accuracy 88.82
Checking accuracy on Test Set Conv 25e
Got 1449 / 1670 with accuracy 86.77


In [ ]:
path = "/content/drive/MyDrive/models/model_6_30e_conv.pt"
model.load_state_dict(torch.load(path))
model.eval()

print_accuracy(dataloaders, model, "Conv 30e")

Checking accuracy on Training Set - Conv 30e
Got 3070 / 3463 with accuracy 88.65
Checking accuracy on Test Set Conv 30e
Got 1448 / 1670 with accuracy 86.71


In [ ]:
path = "/content/drive/MyDrive/models/model_6_25e_ft.pt"
model.load_state_dict(torch.load(path))
model.eval()

# print_accuracy(dataloaders, model, "FT 25e")

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
path = "/content/drive/MyDrive/models/model_6_5e.pt"
model.load_state_dict(torch.load(path))
model.eval()

print_accuracy(dataloaders, model, "FT 5e")

Checking accuracy on Training Set - FT 5e
Got 3149 / 3463 with accuracy 90.93
Checking accuracy on Test Set FT 5e
Got 1453 / 1670 with accuracy 87.01


In [ ]:
path = "/content/drive/MyDrive/models/model_6_60e_conv.pt"
model.load_state_dict(torch.load(path))
model.eval()

print_accuracy(dataloaders, model, "60 Conv")

Checking accuracy on Training Set - 60 Conv
Got 3121 / 3463 with accuracy 90.12
Checking accuracy on Test Set 60 Conv
Got 1462 / 1670 with accuracy 87.54


In [ ]:
class_correct = list(0. for i in range(20))
class_total = list(0. for i in range(20))
with torch.no_grad():
    for data in dataloaders['val']:
        images, labels = data
        batch_size = labels.size(0)

        images = images.cuda()
        labels = labels.cuda()
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(batch_size):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(20):
    # print(class_total)
    print('Accuracy of %5s : %2d %%' % (class_names[i], float(100) * class_correct[i] / class_total[i]))
    # print(f"Accuracy of {classes[i]} : {float(100) * class_correct[i] / class_total[i]}"")

Accuracy of achnabrech : 82 %
Accuracy of baluachraig : 80 %
Accuracy of blanes : 72 %
Accuracy of bothwell : 86 %
Accuracy of caerlaverock : 89 %
Accuracy of  carn :  0 %
Accuracy of clava : 93 %
Accuracy of  doon : 75 %
Accuracy of drumtroddanc : 66 %
Accuracy of drumtroddans : 95 %
Accuracy of kinniel : 100 %
Accuracy of lochranza : 96 %
Accuracy of machrie : 96 %
Accuracy of maiden : 98 %
Accuracy of  ness : 97 %
Accuracy of rothesay : 71 %
Accuracy of semple : 89 %
Accuracy of tantallon : 89 %
Accuracy of tarves : 90 %
Accuracy of tealing : 95 %
